as_default

Returns a context manager that makes this Graph the default graph.
Use this method with the with keyword to specify that ops created within the scope of a block should be added to this graph. In this case, once the scope of the with is exited, the previous default graph is set again as default. There is a stack, so it's ok to have multiple nested levels of as_default calls.

In [5]:
import os
import urllib.request
import tarfile
import tensorflow as tf
import numpy as np
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from grabscreen import grab_screen
import keys as k
import time

keys=k.Keys({})

MODEL_NAME = 'ssd_mobilenet_v3_large_coco_2020_01_14'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90

def download_and_extract_model():
    if not os.path.exists(PATH_TO_CKPT):
        opener = urllib.request.URLopener()
        opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
        tar_file = tarfile.open(MODEL_FILE)
        for file in tar_file.getmembers():
            file_name = os.path.basename(file.name)
            if 'frozen_inference_graph.pb' in file_name:
                tar_file.extract(file, os.getcwd())
        print("Model extracted.")
    else:
        print("Model already exists. No need to extract.")

download_and_extract_model()

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_width, im_height, 3)).astype(np.uint8)

WIDTH = 480
HEIGHT = 270

def determine_movement(mid_x, mid_y, width=WIDTH, height=HEIGHT-15):
    #the center of the screen : 0.5, 0.5
    #the center of the object : mid_x, mid_y
    x_move = 0.5 - mid_x  # the distance from the center of the object
    y_move = 0.5 - mid_y

    #how much to move
    hm_x = x_move / 0.5
    hm_y = y_move / 0.5

    # Move the mouse based on the calculations 
        #decimal to integer
                                                                        #(X, Y)
                                                                        #the reason why negative is flipping the sign of both tof values
                                                                        #that should be where remove the mouse
    keys.keys_worker.SendInput(keys.keys_worker.Mouse(0x000, -1*int(hm_x*width), -1*int(hm_y*width)))

if not os.path.exists(PATH_TO_CKPT):
    print("Model file not found")
else:
    print("Model file found, loading...")

    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.compat.v1.GraphDef()
        with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.70)

Model already exists. No need to extract.
Model file found, loading...


In [10]:
with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph, config=tf.compat.v1.ConfigProto(gpu_options=gpu_options)) as sess:
    stolen = False
    while True:
     
      screen = cv2.resize(grab_screen(region=(0,40,1280,745)), (800,450))
      image_np = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      #x1,y1, x2, y2
      #zero to one
      #boxes[0] : [[3.51673365e-03 4.09537554e-03 9.89943802e-01 1.00000000e+00], ...] 100th, contain the coordinates
      #scores[0] : [0.32287654 0.22373985 0.1553109  0.14345004 0.11081722 0.10204417, ...] 100th
      #classes[0] : [77. 84. 73. 84. 72. 14. ...] 100th
      #num_detections : 100.0 100.0...

      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      
      #check the classes of those boxes
      #if you know the exact size of that object, you could calculate the exact distance of that object
      #measure the amount of pixels in between x1 to x2 that how wide is that
      #from there we can get a relative determination of how far away that object is
      vehicle_dict = {}

      for i,b in enumerate(boxes[0]):
        #https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt
        #                 car                    bus                  truck
        if classes[0][i] == 3 or classes[0][i] == 6 or classes[0][i] == 8:
          if scores[0][i] >= 0.5:

            #where is the middle point of this object
            #middle point : (boxes[0][i][3]+boxes[0][i][1])/2
            mid_x = (boxes[0][i][1]+boxes[0][i][3])/2
            mid_y = (boxes[0][i][0]+boxes[0][i][2])/2

            #the reason why one minus is it'll be smaller as it gets closer
            #the reason why power of 4 is that will give a litt le more granularity
            #height : boxes[0][i][3]-boxes[0][i][1]
            #the reason why round 3 is we are less likely to get duplicate distances.
            apx_distance = round(((1 - (boxes[0][i][3] - boxes[0][i][1]))**4),3) # percentages
            
            #need to put pixel, not percentage, int won't take a float
            #put the approximate relative distance on the car
            #cv2 works in BGR
            cv2.putText(image_np, '{}'.format(apx_distance), (int(mid_x*800),int(mid_y*450)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

            
            if apx_distance <=0.5:
              if mid_x > 0.3 and mid_x < 0.7:
                cv2.putText(image_np, 'WARNING!!!', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)
            

            vehicle_dict[apx_distance] = [mid_x, mid_y, scores[0][i]]

      #Acquiring a Vehicle for the Agent
      #if the agent does not have a vehicle, we want to be able to steal a vehicle : find the vehicle and steal it
      if len(vehicle_dict) > 0:
        closest = sorted(vehicle_dict.keys())[0]
        vehicle_choice = vehicle_dict[closest]
        print('CHOICE:',vehicle_choice)
        if not stolen:
          
          #approach the car
          determine_movement(mid_x = vehicle_choice[0], mid_y = vehicle_choice[1], width=1280, height=705)
          if closest < 0.1:
            keys.directKey("w", keys.key_release)
            keys.directKey("f")
            time.sleep(0.05)          
            keys.directKey("f", keys.key_release)
            stolen = True
          else:
            keys.directKey("w")

      cv2.imshow('window',image_np)
      if cv2.waitKey(25) & 0xFF == ord('q'):
          cv2.destroyAllWindows()
          break

CHOICE: [0.34951069951057434, 0.45252272486686707, 0.5289905]
CHOICE: [0.3493216931819916, 0.45339876413345337, 0.5068966]
CHOICE: [0.3001091778278351, 0.44952696561813354, 0.7527705]
CHOICE: [0.40250617265701294, 0.4670773446559906, 0.52278167]
CHOICE: [0.4506341218948364, 0.463449090719223, 0.57004493]
CHOICE: [0.3068399429321289, 0.46038302779197693, 0.5560017]
CHOICE: [0.30452093482017517, 0.4603217542171478, 0.7109408]
CHOICE: [0.30948692560195923, 0.4572943150997162, 0.5403854]
CHOICE: [0.308949738740921, 0.45391514897346497, 0.54790354]
CHOICE: [0.30574214458465576, 0.45549896359443665, 0.5947436]
CHOICE: [0.30370745062828064, 0.4538889527320862, 0.7568114]
CHOICE: [0.30553963780403137, 0.45161163806915283, 0.6306545]
CHOICE: [0.3049454092979431, 0.4527812600135803, 0.67570335]
CHOICE: [0.30643734335899353, 0.4530981779098511, 0.5211911]
CHOICE: [0.45502349734306335, 0.45357081294059753, 0.51321644]
CHOICE: [0.45378369092941284, 0.45265984535217285, 0.69347197]
CHOICE: [0.454650